In [6]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pickle
import ast

# Load our data with warning suppressed
df_clean = pd.read_csv('data/mtg_cards_clean.csv', low_memory=False)
print(f"Loaded {len(df_clean)} cards")

# Recreate keyword features function
def create_keyword_features(df):
    important_keywords = [
        'defender', 'flying', 'vigilance', 'deathtouch', 'lifelink', 'trample',
        'haste', 'first strike', 'double strike', 'hexproof', 'indestructible',
        'proliferate', 'counter', 'artifact', 'enchantment', 'token', 'draw',
        'graveyard', 'exile', 'sacrifice', 'destroy', 'search', 'toughness',
        'power', 'enters', 'whenever', 'combat damage', 'attacks', 'skulk',
        'lose', 'gain', 'life', 'dies', 'discard', 'attacking', 'triggered',
        'ability', 'additional'
    ]
    
    keyword_matrix = []
    for _, row in df.iterrows():
        text = row['combined_text'].lower()
        features = []
        for keyword in important_keywords:
            count = text.count(keyword)
            features.append(min(count, 3))
        keyword_matrix.append(features)
    
    return np.array(keyword_matrix), important_keywords

# Color identity functions
def get_color_identity(colors_string):
    if pd.isna(colors_string) or colors_string == '[]':
        return set()
    try:
        colors_list = ast.literal_eval(colors_string)
        return set(colors_list) if colors_list else set()
    except:
        return set()

def is_legal_in_deck(card_colors_string, commander_colors_string):
    card_identity = get_color_identity(card_colors_string)
    commander_identity = get_color_identity(commander_colors_string)
    return card_identity.issubset(commander_identity)

# Create the model components
keyword_matrix, keywords = create_keyword_features(df_clean)

# Save everything
model_data = {
    'df_clean': df_clean,
    'keyword_matrix': keyword_matrix,
    'keywords': keywords,
    'get_color_identity': get_color_identity,
    'is_legal_in_deck': is_legal_in_deck
}

with open('data/mtg_model.pkl', 'wb') as f:
    pickle.dump(model_data, f)

print("✅ Complete model with color identity saved successfully!")

Loaded 27623 cards
✅ Complete model with color identity saved successfully!


In [7]:
# Create corrected Flask app without pickled functions
flask_app_code = '''
from flask import Flask, render_template, request, jsonify
import pickle
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import ast

app = Flask(__name__)

# Define color functions directly in this file
def get_color_identity(colors_string):
    """Convert color string to set for easier comparison"""
    if pd.isna(colors_string) or colors_string == '[]':
        return set()
    try:
        colors_list = ast.literal_eval(colors_string)
        return set(colors_list) if colors_list else set()
    except:
        return set()

def is_legal_in_deck(card_colors_string, commander_colors_string):
    """Check if card is legal in commander's color identity"""
    card_identity = get_color_identity(card_colors_string)
    commander_identity = get_color_identity(commander_colors_string)
    return card_identity.issubset(commander_identity)

# Load our saved model (without the functions)
print("Loading ML model...")
with open('data/mtg_model.pkl', 'rb') as f:
    model_data = pickle.load(f)

df_clean = model_data['df_clean']
keyword_matrix = model_data['keyword_matrix']
keywords = model_data['keywords']

print(f"Model loaded! {len(df_clean)} cards available")

def find_recommendations(commander_name, num_recommendations=10):
    """Main recommendation function for web app"""
    # Find commander
    card_matches = df_clean[df_clean['name'].str.contains(commander_name, case=False, na=False)]
    
    if len(card_matches) == 0:
        return {"error": f"Commander '{commander_name}' not found"}
    
    commander_row = card_matches.iloc[0]
    commander_idx = card_matches.index[0]
    commander_colors = commander_row['colors']
    commander_vector = keyword_matrix[commander_idx].reshape(1, -1)
    
    # Calculate similarities
    similarities = cosine_similarity(commander_vector, keyword_matrix).flatten()
    all_indices = similarities.argsort()[::-1]
    
    # Filter for legal cards
    results = []
    for idx in all_indices:
        if idx == commander_idx:
            continue
            
        card_row = df_clean.iloc[idx]
        if is_legal_in_deck(card_row['colors'], commander_colors):
            results.append({
                'name': card_row['name'],
                'similarity': float(similarities[idx]),
                'type': card_row['type_line'],
                'colors': list(get_color_identity(card_row['colors'])),
                'text': card_row['oracle_text'][:200] + "..." if len(card_row['oracle_text']) > 200 else card_row['oracle_text'],
                'price': card_row.get('usd', 'N/A')
            })
            
            if len(results) >= num_recommendations:
                break
    
    return {
        "commander": {
            "name": commander_row['name'],
            "colors": list(get_color_identity(commander_colors)),
            "type": commander_row['type_line']
        },
        "recommendations": results
    }

@app.route('/')
def home():
    return render_template('index.html')

@app.route('/recommend', methods=['POST'])
def recommend():
    data = request.get_json()
    commander = data.get('commander', '')
    num_recs = data.get('num_recommendations', 10)
    
    result = find_recommendations(commander, num_recs)
    return jsonify(result)

if __name__ == '__main__':
    app.run(debug=True, port=5000)
'''

# Save the corrected Flask app
with open('app.py', 'w') as f:
    f.write(flask_app_code)

print("✅ Fixed Flask app created!")
print("Now try running: python app.py")

✅ Fixed Flask app created!
Now try running: python app.py


In [4]:
import os

# Create templates directory
os.makedirs('templates', exist_ok=True)

# Create the HTML template
html_template = '''
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>MTG Commander Deck Recommender</title>
    <style>
        body { font-family: Arial, sans-serif; max-width: 1200px; margin: 0 auto; padding: 20px; }
        .header { text-align: center; margin-bottom: 30px; }
        .search-box { margin-bottom: 30px; }
        .search-box input { padding: 10px; width: 300px; font-size: 16px; }
        .search-box button { padding: 10px 20px; font-size: 16px; background: #007bff; color: white; border: none; cursor: pointer; }
        .commander-info { background: #f8f9fa; padding: 15px; border-radius: 8px; margin-bottom: 20px; }
        .recommendations { display: grid; gap: 15px; }
        .card { border: 1px solid #ddd; padding: 15px; border-radius: 8px; background: white; }
        .card-name { font-weight: bold; font-size: 18px; color: #007bff; }
        .card-type { color: #666; margin: 5px 0; }
        .card-text { margin: 10px 0; line-height: 1.4; }
        .similarity { background: #28a745; color: white; padding: 3px 8px; border-radius: 4px; font-size: 12px; }
        .colors { margin: 5px 0; }
        .color { display: inline-block; width: 20px; height: 20px; border-radius: 50%; margin-right: 5px; }
        .loading { text-align: center; font-style: italic; color: #666; }
        .error { color: #dc3545; background: #f8d7da; padding: 10px; border-radius: 4px; }
    </style>
</head>
<body>
    <div class="header">
        <h1>🧙‍♂️ MTG Commander Deck Recommender</h1>
        <p>Enter a commander name to get AI-powered card recommendations with perfect color identity matching!</p>
    </div>

    <div class="search-box">
        <input type="text" id="commanderInput" placeholder="Enter commander name (e.g., Arcades)" />
        <button onclick="getRecommendations()">Get Recommendations</button>
    </div>

    <div id="results"></div>

    <script>
        async function getRecommendations() {
            const commander = document.getElementById('commanderInput').value;
            const resultsDiv = document.getElementById('results');
            
            if (!commander.trim()) {
                resultsDiv.innerHTML = '<div class="error">Please enter a commander name</div>';
                return;
            }
            
            resultsDiv.innerHTML = '<div class="loading">🔍 Searching for recommendations...</div>';
            
            try {
                const response = await fetch('/recommend', {
                    method: 'POST',
                    headers: { 'Content-Type': 'application/json' },
                    body: JSON.stringify({ commander: commander, num_recommendations: 10 })
                });
                
                const data = await response.json();
                
                if (data.error) {
                    resultsDiv.innerHTML = `<div class="error">${data.error}</div>`;
                    return;
                }
                
                displayResults(data);
            } catch (error) {
                resultsDiv.innerHTML = '<div class="error">Error getting recommendations. Please try again.</div>';
            }
        }
        
        function displayResults(data) {
            const resultsDiv = document.getElementById('results');
            
            let html = `
                <div class="commander-info">
                    <h2>Commander: ${data.commander.name}</h2>
                    <p><strong>Type:</strong> ${data.commander.type}</p>
                    <p><strong>Color Identity:</strong> ${data.commander.colors.join(', ') || 'Colorless'}</p>
                </div>
                
                <h3>🎯 Recommended Cards (${data.recommendations.length} found)</h3>
                <div class="recommendations">
            `;
            
            data.recommendations.forEach((card, index) => {
                html += `
                    <div class="card">
                        <div class="card-name">${index + 1}. ${card.name} 
                            <span class="similarity">Similarity: ${(card.similarity * 100).toFixed(1)}%</span>
                        </div>
                        <div class="card-type">${card.type}</div>
                        <div class="colors">Colors: ${card.colors.join(', ') || 'Colorless'}</div>
                        <div class="card-text">${card.text}</div>
                    </div>
                `;
            });
            
            html += '</div>';
            resultsDiv.innerHTML = html;
        }
        
        // Allow Enter key to trigger search
        document.getElementById('commanderInput').addEventListener('keypress', function(e) {
            if (e.key === 'Enter') {
                getRecommendations();
            }
        });
    </script>
</body>
</html>
'''

# Save the HTML template
with open('templates/index.html', 'w') as f:
    f.write(html_template)

print("✅ HTML template created in 'templates/index.html'")
print("\nReady to run the web app! 🚀")

✅ HTML template created in 'templates/index.html'

Ready to run the web app! 🚀


In [1]:
# Let's create this in a Jupyter notebook cell for better formatting control
readme_content = '''# 🧙‍♂️ MTG Commander Deck Recommender

An AI-powered Magic: The Gathering Commander deck recommendation system that suggests cards with high synergy while respecting color identity rules.

## ✨ Features

- **Content-Based Filtering**: Uses machine learning to find cards with similar mechanics and text
- **Color Identity Compliance**: Only recommends cards legal in your commander's color identity  
- **Web Interface**: Clean, user-friendly web app for easy card discovery
- **Budget Conscious**: Focuses on cards under $10
- **Keyword-Based Similarity**: Matches on important MTG mechanics like proliferate, defender, artifacts, etc.

## 🚀 How It Works

1. **Data Collection**: Gathers 4,000+ Commander-legal cards under $10 from Scryfall API
2. **Feature Extraction**: Converts card text into numerical features based on MTG keywords
3. **Similarity Calculation**: Uses cosine similarity to find mechanically similar cards
4. **Color Filtering**: Ensures all recommendations are legal in the commander's color identity
5. **Web Interface**: Presents results in an intuitive web application

## 🛠️ Technology Stack

- **Python 3.11**: Core programming language
- **scikit-learn**: Machine learning algorithms
- **TensorFlow**: ML framework (with Apple Silicon optimization)
- **Flask**: Web framework
- **Pandas**: Data manipulation
- **Scryfall API**: MTG card data source

## 🏃‍♂️ Quick Start

1. **Clone the repository**
   ```bash
   git clone https://github.com/yourusername/mtg-commander-recommender.git
   cd mtg-commander-recommender

2. **Set up Python environment**
   python3.11 -m venv mtg-env
   source mtg-env/bin/activate  # On Windows: mtg-env\\Scripts\\activate
   pip install -r requirements.txt

3. **Run data collection**
   jupyter notebook
   # Run 01_data_exploration.ipynb to collect fresh data

4. **Start app**
   python app.py

5. **Run app**
   Open your browser to http://localhost:5000

## 🧠 ML Approach
This project uses content-based filtering to recommend cards by analyzing MTG-specific keywords and mechanics rather than generic text similarity.

## 🙏 Acknowledgments

Scryfall API for MTG data
The MTG community for deck building insights

Magic: The Gathering is © Wizards of the Coast
'''

In [3]:
with open('README.md', 'w') as f:
    f.write(readme_content)
print("✅ README.md created successfully!")

✅ README.md created successfully!
